# BM25 model

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from funcs import load_datasets, get_docs
from ir_measures import measures, calc_aggregate

In [2]:
datasets = load_datasets(["ru", "zh", "fa"])

In [3]:
# Load the Qrels and Queries
qrels = pd.DataFrame(datasets["ru"].qrels_iter())  # ground truth
queries = pd.DataFrame(datasets["ru"].queries_iter())  # queries
documents = pd.DataFrame(datasets["ru"].docs_iter())  # documents

In [4]:
common_query_ids = set(qrels["query_id"]).intersection(queries["query_id"])
filtered_queries = queries[queries["query_id"].isin(common_query_ids)]

In [5]:
#tokenize and normalize Russian

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

#nltk.download('punkt')
#nltk.download('stopwords')

def preprocess_ru(text):
    # Tokenize
    tokens = word_tokenize(text)
    # Lowercase and remove punctuation
    tokens = [word.lower() for word in tokens if word.isalpha()]
    # Remove stop words
    stop_words = set(stopwords.words('russian'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)  # Join tokens back into a single string

In [6]:
# Apply preprocessing to the document text
documents['processed_text'] = documents['text'].apply(preprocess_ru)

In [7]:
# Convert the processed documents to a list
processed_documents = documents["processed_text"].tolist()

# Verify a few processed documents
print(processed_documents[:5])

['двое друзей встретились парке гуляя собаками предложил зайти позавтракать ближайшее кафе пустят туда собаками возразил второй первый решительно направился кафе своей немецкой овчаркой хозяин остановил словами сэр нам заходить животными слепой это хозяин извинился проводил собакой столику друг подождал улице пять минут попробовал сделать самое ваш поводырь чихуахуа скептически осведомился хозяин чихуахуа удивился мужчина подсунули анекдот', 'нашли ошибку текст который выделяем смотрим выделили слишком максимальное количество символов попробуйте снова спасибо сообщение отправлено скоро исправим', 'бежит мышка кота прыгает стола попадает бутылку недопитым вином стоящую полу барахтается говорит коту вытащи дай умереть кошмарной смертью убежишь честное слово вытащил первым делом норку шасть сидит кот обижается мышка выходи сказала убежишь мало сказать мужчине женщина нетрезвом виде анекдот', 'председатель федерального ведомства охране конституции германии масен maaßen отправлен отставку о

In [8]:
# Apply preprocessing to the 'title' column

filtered_queries["processed_query"] = filtered_queries["mt_title"].apply(preprocess_ru) # change description to title to use the title column

# Convert the processed queries to a list
processed_queries = filtered_queries["processed_query"].tolist()

# Verify the processed queries
print(processed_queries[:5])

['британские королевские новости влияют', 'суверенитет гибралтара брексита', 'торговое соглашение сша южной кореей', 'северокорейские землетрясения ядерные испытания', 'кораблекрушения историческая европейская торговля']


In [9]:
#Vectorize Queries and Documents with TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform on the combined data for consistent vocabulary
tfidf_documents = vectorizer.fit_transform(processed_documents)
tfidf_queries = vectorizer.transform(processed_queries)


In [10]:
from rank_bm25 import BM25Okapi

In [11]:
# Initialize the BM25 model with the tokenized corpus
bm25 = BM25Okapi(processed_documents)

In [12]:
# Example query: Retrieve top documents for each query
results = {}
for idx, query in filtered_queries.iterrows():
    query_tokens = query['processed_query']
    doc_scores = bm25.get_scores(query_tokens)
    top_n_indices = doc_scores.argsort()[-10:][::-1]  # Top 10 documents
    results[query['query_id']] = documents.iloc[top_n_indices][['doc_id', 'title', 'text']]

In [13]:
# View results for a specific query_id
query_id_to_view = filtered_queries['query_id'].iloc[0]
print(f"Top documents for query {query_id_to_view}:")
print(results[query_id_to_view])

Top documents for query 3:
                                      doc_id  \
515510  207cb3a5-a2be-42d7-86cf-530120d0d570   
801219  e2f3d3d3-7b6f-47df-96e6-31567aa69707   
369558  dd659748-b8f2-4e5e-b887-76e39840b74c   
76212   48f5226f-46e8-4c66-81eb-20526043e274   
209094  75083127-54db-4c86-b622-4150377d80ff   
842207  81fd399d-f667-46b0-8011-33d8442b7cb1   
336776  22e77ebd-1baf-407a-a7b9-8349d97f17d6   
317190  e1247479-27ef-43f7-bbd9-047555c68c2b   
340838  a747c84f-791b-4a7d-8c7a-79c8063aac72   
507202  a2c7a8c6-0146-4f1e-b52b-94a6df221a59   

                                                    title  \
515510  Валюту под матрас: российские банки теряли при...   
801219  Массовый сход в сквере в центре Екатеринбурга....   
369558                                 НБУ ”потерял” курс   
76212   Ящик пандоры – Сергей Глазьев об экономике Кры...   
209094   Революция как вызов истории, - Борис Кагарлицкий   
842207  Летом к морю в Украине запустили 30 летних пое...   
336776  Модный ди

In [14]:
bm25_run = []

# Convert BM25 results into the required format
for query_id, docs in results.items():
    # Compute BM25 scores for the query
    query_tokens = filtered_queries.loc[filtered_queries["query_id"] == query_id, "processed_query"].values[0]
    doc_scores = bm25.get_scores(query_tokens)
    
    # Iterate over the retrieved documents
    for doc_index in docs.index:
        doc_id = docs.at[doc_index, "doc_id"]
        score = doc_scores[documents.index[documents["doc_id"] == doc_id][0]]  # Map the score by index
        bm25_run.append({'query_id': query_id, 'doc_id': doc_id, 'score': score})

In [15]:
your_run = pd.DataFrame(bm25_run)

print("BM25 Run DataFrame:\n", your_run.head())

BM25 Run DataFrame:
   query_id                                doc_id       score
0        3  207cb3a5-a2be-42d7-86cf-530120d0d570  228.582652
1        3  e2f3d3d3-7b6f-47df-96e6-31567aa69707  228.567458
2        3  dd659748-b8f2-4e5e-b887-76e39840b74c  228.558398
3        3  48f5226f-46e8-4c66-81eb-20526043e274  228.553375
4        3  75083127-54db-4c86-b622-4150377d80ff  228.546907


In [16]:
# Calculate the average score across all rows
average_score = your_run["score"].mean()

# Print the result
print(f"The average score across all query-document pairs is: {average_score:.4f}")

The average score across all query-document pairs is: 231.5656


BM25 Evaluation


In [17]:
import ir_measures
from ir_measures import nDCG, P, Judged, RBP, AP, RR, R

evaluation_metrics = ir_measures.calc_aggregate(
    [
        nDCG@20,  # Normalized Discounted Cumulative Gain @20
        P@5,  # Precision @5
        P(rel=1)@5,  # Precision for relevance level >=1 @5
        Judged@10,  # Judged documents @10
        R@100,  # Recall @100
        R@1000,  # Recall @1000
        AP,  # Average Precision
        RR@10,  # Reciprocal Rank @10
    ],
    qrels,
    your_run
)
print("Results for BM25: ",evaluation_metrics)


Results for BM25:  {R@100: 0.0, RR@10: 0.0, AP: 0.0, R@1000: 0.0, Judged@10: 0.0, nDCG@20: 0.0, P@5: 0.0}


In [23]:
# Debugging why we have zeros

# Inspect overlapping query-doc pairs
overlap = pd.merge(your_run[['query_id', 'doc_id']], qrels[['query_id', 'doc_id']], on=['query_id', 'doc_id'], how='inner')
if overlap.empty:
    print("No overlapping query-doc pairs between run and qrels.")
else:
    print(f"Found {len(overlap)} overlapping entries.")
    print(overlap)


No overlapping query-doc pairs between run and qrels.
